In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as f

In [ ]:
class MLP(nn.Module):
    def __init__(self, architecture, activation, dropout_p=0.0):
        super()__init__() # chama o construtor da classe pai.
        self.layers = nn.ModuleList()
        self.activation_name = activation
        self.dropout_p = dropout_p

        for i in range(len(architecture) - 1):
            self.layers.append(nn.Linear(architecture[i], architecture[i+1]))

    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            x = layer(x)
            if self.activation_name == 'relu':
                x = F.relu(x)
            elif self.activation_name == 'tanh':
                x = torch.tanh(x)
            if self.dropout_p > 0:
                x = F.dropout(x, p=self.dropout_p, training=self.training)
        x = self.layers[-1](x)  # última camada (sem ativação)
        return x


# retorna um número entre 0 e 1 representando o quão bom foi. 0 = pior resultado (muito longe), 1 = acertou
def binary_accuracy(preds, labels, threshold=0.5):
    probs = torch.sigmoid(preds)
    preds_class = (probs > threshold).float()
    return (preds_class == labels).float().mean().item()

# Retorna a proporção de exemplos que a rede previu corretamente.
def multiclass_accuracy(preds, labels):
    preds_class = preds.argmax(dim=1)                # escolhe classe com maior logit
    return (preds_class == labels).float().mean().item()




In [ ]:
def train_model_with_validation(model, optimizer, criterion, X_train, y_train, X_val, y_val, epochs, patience):
    best_loss = float('inf')
    best_model = None
    patience_counter = 0

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):

        # Treinamento
        model.train() # Coloca a rede em modo treinamento. (ativa dropout, etc)
        outputs = model(X_train) # faz o forward pass — aplica as camadas da rede nos dados de treino.
        loss = criterion(outputs, y_train) # calcula a função de perda com as saídas da rede e os rótulos reais.
        optimizer.zero_grad() # zera os gradientes antigos acumulados (importante!).
        loss.backward() # calcula os gradientes da perda em relação aos pesos (backpropagation).
        optimizer.step() # atualiza os pesos da rede com base nos gradientes.

        # Avaliação
        model.eval() # coloca a rede em modo avaliação (desativa dropout, etc).
        with torch.no_grad(): # desativa o rastreamento de gradientes — melhora a performance e reduz uso de memória.
            train_acc = accuracy(outputs, y_train) # Mede a acurácia no conjunto de treinamento
            
            if X_val and y_val:
                val_outputs = model(X_val) # Obtém os outputs do conjunto de validação.
                val_loss = criterion(val_outputs, y_val) # Calcula novamente a perda, agora com os dados de validação.
                val_acc = accuracy(val_outputs, y_val) # Mede a acurácia no conjunto de validação.

        # Armazena as métricas obtidas pelo treinamento/validação no dicionário history.
        history['train_loss'].append(loss.item())
        history['train_acc'].append(train_acc)

        if X_val and y_val:
            history['val_loss'].append(val_loss.item())
            history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

        # Early stopping - with validation set
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    # Restaura o melhor modelo salvo.
    model.load_state_dict(best_model)
    return model, history

def train_model_no_validation(model, optimizer, criterion, X_train, y_train, epochs, patience):
    best_loss = float('inf')
    best_model = None
    patience_counter = 0

    history = {'train_loss': [], 'train_acc': []}

    for epoch in range(epochs):

        # Treinamento
        model.train()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Avaliação (em modo eval, mas nos mesmos dados de treino)
        model.eval()
        with torch.no_grad():
            train_acc = accuracy(outputs, y_train)

        # Registro
        history['train_loss'].append(loss.item())
        history['train_acc'].append(train_acc)

        print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Train Acc: {train_acc:.4f}")

        # Early stopping baseado em perda de treino
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    model.load_state_dict(best_model)
    return model, history



In [ ]:
dataset_treinamento = '..\\perceptron_datasets\\train_dataset1.csv'
dataset_testes = '..\\perceptron_datasets\\test_dataset1.csv'

df_train_loaded = pd.read_csv(dataset_treinamento)
df_test_loaded = pd.read_csv(dataset_testes)

x_train = df_train_loaded.drop('label', axis=1).values
y_train = df_train_loaded['label'].values

x_test = df_test_loaded.drop('label', axis=1).values
y_test = df_test_loaded['label'].values

#### Experimento 1
Camadas = 2, 3, 1

Função de perda (ou de custo): Mede o quão ruim está a previsão da rede em relação à resposta esperada. Calcula o erro da rede. É calculado no forward e usado no backpropagation para ajustar os pesos. Utilizando: Binary cross entropy (medida antes da função de ativação da saída)

Patience: A partir de quantas iterações sem melhoria do erro eu devo parar

In [ ]:
camadas = [2, 3, 1]
fx_ativacao = 'relu'
probabilidade_dropout = 0.0 # sem dropout

modelo = MLP(camadas, fx_ativacao, probabilidade_dropout)